#### Includes

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import altair as alt
from altair import datum
import pandas as pd
import time
import os.path as P
from pytorch_lightning.utilities.cloud_io import load as pl_load
import pandas as pd
import altair as alt
alt.data_transformers.disable_max_rows()
None

In [3]:
from sources import load_dataset, default_config, parse_config
from run_train import setup_model
from src.trainer import BinaryClassifierNet

In [4]:
from steered_cnn.utils.rotequivariance_toolbox import *
from steered_cnn.steered_conv.steerable_filters import radial_steerable_filter, plot_filter
from steered_cnn.steered_conv import SteerableKernelBase

## Initialization

In [5]:
RUN_ID = '8/dd8fc55431b5441e811d33871cd9aca6'
OLD = False

In [6]:
art_path = P.abspath(P.join('/home/gaby/mlflow/mlruns', RUN_ID, 'artifacts/'))
cfg = parse_config(art_path+'/cfg_extended.yaml')
cfg['script-arguments']['gpus']='0'

In [7]:
model = setup_model(cfg.model, old=OLD)
net = BinaryClassifierNet(model=model)
state_dict = pl_load(art_path+'/best-val-roc.ckpt'.format(25), map_location='cuda:0')['state_dict']
net.load_state_dict(state_dict)
net.eval().cuda()
None

## Weight Value Analysis

In [8]:
base = model.base
weights = [_ for _ in model.parameters() if (_.ndim==3 and _.shape[-1]==len(base.kernels_label))]

In [9]:
W = [_ for _ in model.parameters() if (_.ndim==3 and _.shape[-1]==len(base.kernels_label))]

In [10]:
base.plot_weights_hist(W, binspace='log', wrange=(1e-5,1), bins=1000)

alt.Chart(...)

In [11]:
W = base.create_weights(100,100)
angle, infos = base.flatten_weights(W, complex=False)
infos

[{'r_name': 'r=0 ', 'name': 'k=0 r=0 ', 'k': 0, 'r': 0},
 {'r_name': 'r=1 ', 'name': 'k=0 r=1 ', 'k': 0, 'r': 1},
 {'r_name': 'r=2 ', 'name': 'k=0 r=2 ', 'k': 0, 'r': 2},
 {'r_name': 'r=3 ', 'name': 'k=0 r=3 ', 'k': 0, 'r': 3},
 {'r_name': 'r=1 R', 'name': 'k=1 r=1 R', 'k': 1, 'r': 1},
 {'r_name': 'r=2 R', 'name': 'k=1 r=2 R', 'k': 1, 'r': 2},
 {'r_name': 'r=3 R', 'name': 'k=1 r=3 R', 'k': 1, 'r': 3},
 {'r_name': 'r=1 R', 'name': 'k=2 r=1 R', 'k': 2, 'r': 1},
 {'r_name': 'r=2 R', 'name': 'k=2 r=2 R', 'k': 2, 'r': 2},
 {'r_name': 'r=3 R', 'name': 'k=2 r=3 R', 'k': 2, 'r': 3},
 {'r_name': 'r=2 R', 'name': 'k=3 r=2 R', 'k': 3, 'r': 2},
 {'r_name': 'r=3 R', 'name': 'k=3 r=3 R', 'k': 3, 'r': 3},
 {'r_name': 'r=2 R', 'name': 'k=4 r=2 R', 'k': 4, 'r': 2},
 {'r_name': 'r=3 R', 'name': 'k=4 r=3 R', 'k': 4, 'r': 3},
 {'r_name': 'r=2 R', 'name': 'k=5 r=2 R', 'k': 5, 'r': 2},
 {'r_name': 'r=3 R', 'name': 'k=5 r=3 R', 'k': 5, 'r': 3},
 {'r_name': 'r=1  I', 'name': 'k=1 r=1  I', 'k': 1, 'r': 1},
 {'

In [12]:
import math
n_out, n_in = 10,10
angle = base.expand_r(torch.rand((n_out, n_in,base.k_len))*(2*math.pi), dim=2)


In [13]:
base.plot_weights_dist(weights, complex='angle')

alt.FacetChart(...)

In [14]:
base.plot_weights_dist(base.create_weights(100,500))

alt.FacetChart(...)

## Gradient Analysis

In [15]:
trainD, validD, testD = load_dataset(cfg)

/home/gaby/.conda/envs/nnet/lib/python3.7/site-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [16]:
net.train()
net.zero_grad()
for batch in trainD:
    batch_cuda = {k: v.cuda() for k, v in batch.items()}
    loss = net.training_step(batch_cuda, 0)
    loss.backward()
    break

In [17]:
weights_grad = [w.grad for w in weights]
base.plot_weights_dist(weights_grad)

alt.FacetChart(...)

In [18]:
base.plot_weights_hist(weights_grad, complex='norm', wrange=(1e-11, 1e-2), binspace='log')

alt.Chart(...)